In [3]:
from sed2.processes import sed_process_registry
from sed2.core import (
    serialize_instance, generate_script, ports, annotate, register, generate_composite_process, get_process_schema)
from bigraph_viz import plot_bigraph, plot_flow, pf
import numpy as np

In [6]:
# add sed processes to global namespace
sed_process_registry.activate_all(globals())
print(pf(sed_process_registry.registry))

{ 'data_description': { 'annotation': 'sed:data_description',
                        'ports': { 'inputs': { 'data_file': 'str',
                                               'file_format': 'str'},
                                   'outputs': {'data': 'pd.DataFrame'}},
                        'process': <function data_description at 0x15f9a0940>},
  'is_steady_state': { 'annotation': 'sed:is_steady_state',
                       'ports': { 'inputs': { 'model': 'Model',
                                              'tolerance': 'float'},
                                  'outputs': {'sum': 'float'}},
                       'process': <function is_steady_state at 0x15f9a0b80>},
  'loop': { 'annotation': 'sed:loop',
            'ports': { 'inputs': { 'number_of_steps': 'int',
                                   'start': 'float',
                                   'step': 'float'},
                       'outputs': {}},
            'process': <function loop at 0x16a066a60>},
  'model_get_

In [7]:
# SED requires some more primitive operations for this

@ports({
    'inputs': {
        'start': 'float',
        'number_of_steps': 'int',
        'step': 'float'},
    'outputs': {}})
@annotate('sed:loop')
def loop(start=0.001, number_of_steps=9, step=0.001):
    pass

@ports({
    'outputs': {
        'value': 'float'}})
@annotate('sed:loop')
def random_number():
    return random.random()

# add to the registry
functions = [
    loop,
    random_number,
]

sed_process_registry = register(functions, sed_process_registry)
# print(pf(sed_process_registry.registry))

```
# This experiment randomizes two parameters, and
# computes the control coefficient for
# every reaction step in the model
# Outputs a 2D array, first and second column are k1 and k2
# remaining columns are the control coefficients
result1 is 1D array
result2 = is 1D array
trials = 10
load model (xyz.xml)  # This experiment only applies to SBML models
configure simulator (....)
for i = 0 to trials - 1 do
     model.k1 = random()
     model.k2 = random()
     for j = 0 to numReactions - 1 do
         compute Steady state
         if steadystate found:
            result1[i][0] = model.k1
            result1[i][1] = model.k2
            result1[i][j+2] = compute control coefficient (J1, j)
            sum = sum + result1[i][i+2]
          else
            handle error condition in some way,
            eg ignore and continue and record bad result in results array
        result2[i] = sum  # to check that results are ok
output table result1
output table result2
```

In [8]:
import numpy as np
import random

trials = 10
numReactions = 10
result1 = np.empty((trials, numReactions + 2))
result2 = np.empty(trials)
model_path = "susceptible_zombie.xml"

model = sbml_model_from_path(path_to_sbml=model_path)  # Load the model

# configure simulator (...)  # Configure the simulator here

for i in range(trials):
    k1 = random_number()
    k2 = random_number()
    model_set_value(model, 'S', k1)
    model_set_value(model, 'Z', k2)
    
    total = 0
    for j in range(numReactions):
        ss = steady_state_values(model, selection_list=['S', 'Z'])
        
        # Check if steady state is found
        steady_state_found = True  # Replace with actual condition
        if steady_state_found:  
            s = model_get_value(model, 'S')
            z = model_get_value(model, 'Z')
            result1[i][0] = s
            result1[i][1] = z
            # result1[i][j+2] = compute control coefficient (J1, j)
            # Replace the line above with the actual computation
            total += result1[i][j + 2]
        else:
            # handle error condition in some way,
            # eg ignore and continue and record bad result in results array
            pass
    result2[i] = total

print("Result1:")
print(result1)
print("Result2:")
print(result2)

Result1:
[[4.90376534e-001 4.72780626e-001 1.29939265e-321 0.00000000e+000
  0.00000000e+000 0.00000000e+000 0.00000000e+000 0.00000000e+000
  0.00000000e+000 0.00000000e+000 0.00000000e+000 0.00000000e+000]
 [6.36223628e-001 7.52807698e-002 3.00086990e-314 0.00000000e+000
  0.00000000e+000 0.00000000e+000 0.00000000e+000 0.00000000e+000
  0.00000000e+000 0.00000000e+000 0.00000000e+000 0.00000000e+000]
 [7.63746429e-001 7.01344224e-001 0.00000000e+000 0.00000000e+000
  0.00000000e+000 0.00000000e+000 0.00000000e+000 0.00000000e+000
  0.00000000e+000 0.00000000e+000 0.00000000e+000 0.00000000e+000]
 [7.88779509e-001 6.14305784e-001 0.00000000e+000 0.00000000e+000
  0.00000000e+000 0.00000000e+000 0.00000000e+000 0.00000000e+000
  0.00000000e+000 0.00000000e+000 0.00000000e+000 0.00000000e+000]
 [5.35174152e-001 6.35247487e-001 0.00000000e+000 0.00000000e+000
  0.00000000e+000 0.00000000e+000 0.00000000e+000 0.00000000e+000
  0.00000000e+000 0.00000000e+000 0.00000000e+000 0.00000000e+0